In [1]:
excercise_dict={
   "squat":
    {
        
        "knee":
        {
            "extension":"",
            "flexion":"bend",
             "hFlexion":"",
            "abduction":"",
            "angle1":20,
            "angle2":160
            
        }     
        
    },
    "sumosquat":
    {
        
        "knee":
        {
            "extension":"",
            "flexion":"bend",
            "hFlexion":"",
            "abduction":"",
            "angle1":75,
            "angle2":180
            
        }     
        
    },
    
    "sidekick":
    {
        
        "rhip":
        {
            "extension":"",
            "flexion":"",
             "hFlexion":"",
            "abduction":"move",
            "angle1":90,
            "angle2":180
            
        }     
        
    },
     "shoulderpress":
    {
        
        "shoulder":
        {
            "extension":"straighten",
            "flexion":"",
             "hFlexion":"",
            "abduction":"",
            "angle1":90,
            "angle2":180
            
        }     
        
    },
     "benchpress":
    {
        
        "shoulder":
        {
            "extension":"",
            "flexion":"",
             "hflexion":"push",
            "abduction":"",
            "angle1":45,
            "angle2":90
            
        }     
        
    }  
}

In [2]:
for x in excercise_dict["benchpress"]["shoulder"].values():
    print(x)



push

45
90


In [2]:
import pandas as pd

In [29]:
def keygenerator(data):
    angle=data[0]
    joint=data[1]
    excercise=data[2]
    if angle<excercise_dict[excercise][joint]["angle1"] or angle>excercise_dict[excercise][joint]["angle1"]:
        if angle<excercise_dict[excercise][joint]["angle1"]: 
            keywords="straighten "+joint
        else:
            keywords="bend "+joint
    else:
        keywords="good work"
        
    return keywords
    

In [ ]:
def keywordgen(data):
    angle=data[0]
    joint=data[1]
    excercise=data[2]
    if angle>excercise_dict[excercise][joint]["angle1"]:
        keyword= joint+" "+str(excercise_dict[excercise][joint][x] for x in 
                               
    
    

# Loading

In [3]:
import autocompleter 
autocompl = autocompleter.Autocompleter()

In [3]:
df = autocompl.import_json("sample-test.json")
df.shape, df.columns

load json file...
(19, 3)


C:\Users\chari\Text-completion-NLP\autocompleter.py:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  column_as_df = json_normalize(df[column])


((19, 3), Index(['IsFromCustomer', 'Text', 'index'], dtype='object'))

The file contains 22K conversations between a customer and a representative.
For the purpose of this project, we are only interested in completing the threads of the representative.

In [4]:

df.head(10)

,IsFromCustomer,Text,index
0,False,well done continue your excercise,0
1,False,perfectly done continue excercise,0
2,False,good work continue excercise,0
3,False,for bicepcurl angle less for backbone PLease k...,0
4,False,for lunges angle less for backbone PLease kee...,0
5,False,angle less for legs,0
6,False,angle less for hand,0
7,False,angle less for hand PLease keep your arms stra...,0
8,False,angle good for arms,0
9,False,for bicepcurl angle less for neck PLease keep ...,0


In [4]:
df1=pd.read_csv('presentences3.csv')
df1.tail()

,IsFromCustomer,Text,index
998,False,straighten arms upwards,0
999,False,align upwards,0
1000,False,straighten hips sideways,0
1001,False,straighten knees sideways,0
1002,False,good well done,0


# Data Selection and Cleaning

The data is going to separate the threads from the customer and the representative, separate the sentenses based on the punctuation (we will keep the punctuation), the final text will be cleaned up with some light regex and only the sentense larger than 1 word will be kept.

Finally, since the representative has the tendency to ask the same question over and over again, the autocomplete is extremely useful by suggesting a complete sentense. In our case, we will count the number of occurence of the same sentense so we can use it as a feature later on and delete the duplicates.

In [5]:
new_df = autocompl.process_data(df1)
new_df.shape, new_df.columns

select representative threads...
split sentenses on punctuation...
Text Cleaning using simple regex...
calculate nb words of sentenses...
count occurence of sentenses...
remove duplicates (keep last)...
(174, 5)


((174, 5),
 Index(['IsFromCustomer', 'Text', 'index', 'nb_words', 'Counts'], dtype='object'))

# Model and TFIDF matrix

A matrice of similarity is calculated based on the frequency of all the words in the data using tfidfvectorizer

In [6]:
model_tf, tfidf_matrice = autocompl.calc_matrice(new_df)

tfidf_matrice  (174, 291)


# Ranking Function

Finally, the autocomplete is calculating the similarity between the sentense in the data and the prefix of the sentense written by the representative. As a weight feature, we chose to reorder using the frequency of the most common similar sentense.

examples of auto completions

In [9]:
prefix = newgen([160,"hips","sidekick"])
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)[0]

hips straighten sideways      


'Straighten hips sideways'

Now, without any uppercase and just with the important words...

In [10]:
prefix = newgen([90,"knees","squat"])
print(prefix,"     ")
autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)

knees bend down      


['Bend knees down', 'Bend knees backwards', 'Bend knees upwards']

# Online Sources for this project

In [11]:
# https://gist.github.com/jlln/338b4b0b55bd6984f883 modified to keep punctuation
# kaggle google store competition for json read
# https://www.kaggle.com/hamishdickson/weighted-word-autocomplete-using-star-wars-dataset

In [100]:
excercise={
    
    "squat":{
        "backbone":
    {
    "status":2,    
    "key":["","","straighten"],
    "dir":["","",""],
    "angles":[160,180]        
    },
    "knees":
    {
    "status":0,    
    "key":["bend","straighten"],
    "dir":["down","vertically"],
    "angles":[25,160]        
    }
    
    },
    "sidekick":{
        "knees":
    {
    "status":1,    
    "key":["bend","straighten"],
    "dir":["down",""],
    "angles":[160,170]        
    }
    
    }
}

In [101]:
excercise["sidekick"]["knees"]["status"]

1

In [102]:
def newgen(data,statusx):
    angle=data[0]
    joint=data[1]
    exc=data[2]
    if statusx==0:
        if angle>excercise[exc][joint]["angles"][0] and statusx==0:
            keywords=excercise[exc][joint]["key"][statusx]+" "+joint+" "+excercise[exc][joint]["dir"][statusx]     
        else:
            keywords="good work "
            statusx=1
    else:
        if angle<excercise[exc][joint]["angles"][1]:
            keywords=excercise[exc][joint]["key"][statusx]+" "+joint+" "+excercise[exc][joint]["dir"][statusx]
        else:
            keywords="good work"
            statusx=0

            
    return keywords,statusx

    

In [103]:
newgen([90,"knees","squat"],excercise["squat"]["knees"]["status"])

('bend knees down', 0)

In [104]:
l=[180,165,120,90,45,20,10,45,90,120,165,180,165,120,90,45,20,10,45,90,120,165]
print("angle list {}\n".format(l))
print("minimum angle: {} || maximum angle: {}\n".format(excercise["squat"]["knees"]["angles"][0],excercise["squat"]["knees"]["angles"][1]))
status=excercise["squat"]["knees"]["status"]
prev=""
scode=status
print("Exercise name:{} || joint name : {}\n".format("squat","knees"))
for angle in l:
    print("current angle {}".format(angle))
    key,sret=newgen([angle,"knees","squat"],status)
    status=sret
    if prev!=key:
        print("{} status {}".format(key,sret))
    if status!=scode:
        print("direction change")    
    prev=key 
    scode=status

angle list [180, 165, 120, 90, 45, 20, 10, 45, 90, 120, 165, 180, 165, 120, 90, 45, 20, 10, 45, 90, 120, 165]

minimum angle: 25 || maximum angle: 160

Exercise name:squat || joint name : knees

current angle 180
bend knees down status 0
current angle 165
current angle 120
current angle 90
current angle 45
current angle 20
good work  status 1
direction change
current angle 10
straighten knees vertically status 1
current angle 45
current angle 90
current angle 120
current angle 165
good work status 0
direction change
current angle 180
bend knees down status 0
current angle 165
current angle 120
current angle 90
current angle 45
current angle 20
good work  status 1
direction change
current angle 10
straighten knees vertically status 1
current angle 45
current angle 90
current angle 120
current angle 165
good work status 0
direction change


# PHASE 2

In [21]:
def merge(list1, list2):
      
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
    return merged_list

In [22]:
l2=[180,165,120,170,180,170,170,175,170,180,165,180,165,160,170,180,170,170,180,180,170,165]
l1=[180,165,120,90,45,20,10,45,90,120,165,180,165,120,90,45,20,10,45,90,120,165]


In [44]:
lx=merge(l1,l2)
jj=["knees","backbone"]
status=[]    
for x in jj:
    status.append(excercise["squat"][x]["status"])


In [45]:
print(lx)

[(180, 180), (165, 165), (120, 120), (90, 170), (45, 180), (20, 170), (10, 170), (45, 175), (90, 170), (120, 180), (165, 165), (180, 180), (165, 165), (120, 160), (90, 170), (45, 180), (20, 170), (10, 170), (45, 180), (90, 180), (120, 170), (165, 165)]


In [85]:
def newgen(data,statusx):
    angle=data[0]
    joints=data[1]
    exc=data[2]
    i=0
    keywords=""
    for joint in joints:
        if statusx[i]==0:
            if angle[i]>excercise[exc][joint]["angles"][0] and statusx[i]==0:
                keywords+=" "+excercise[exc][joint]["key"][statusx[i]]+" "+joint+" "+excercise[exc][joint]["dir"][statusx[i]]     
            else:
                keywords+="good work "
                statusx[i]=1
        elif statusx[i]==1:
            if angle[i]<excercise[exc][joint]["angles"][1]:
                keywords+=" "+excercise[exc][joint]["key"][statusx[i]]+" "+joint+" "+excercise[exc][joint]["dir"][statusx[i]]
            else:
                keywords+="good work"
                statusx[i]=0
        else:
            if  not angle[i] in range(excercise[exc][joint]["angles"][0],excercise[exc][joint]["angles"][1]+1):
                keywords+=" "+excercise[exc][joint]["key"][statusx[i]]+" "+joint+" "+excercise[exc][joint]["dir"][statusx[i]]
            
            
        i=i+1
    print(statusx)        
    return keywords,statusx

    

In [99]:
print("angle list {}\n".format(l))
print("minimum angle: {} || maximum angle: {}\n".format(excercise["squat"]["knees"]["angles"][0],excercise["squat"]["knees"]["angles"][1]))
status=[]    
for x in jj:
    status.append(excercise["squat"][x]["status"])
prev=""
scode=status
print("Exercise name:{} || joint name : {}\n".format("squat","knees"))
for an in lx:
    print("current angle {}".format(an))
    key,sret=newgen([list(an),jj,"squat"],status)
    status=sret
    if prev!=key:
        print("{} status {}".format(key,sret))
#     if status!=scode:
#         print("direction change") 
    prev=key 
    scode=status

angle list [180, 165, 120, 90, 45, 20, 10, 45, 90, 120, 165, 180, 165, 120, 90, 45, 20, 10, 45, 90, 120, 165]

minimum angle: 25 || maximum angle: 160

Exercise name:squat || joint name : knees

current angle (180, 180)
[0, 2]
 bend knees down status [0, 2]
current angle (165, 165)
[0, 2]
current angle (120, 120)
[0, 2]
 bend knees down straighten backbone  status [0, 2]
current angle (90, 170)
[0, 2]
 bend knees down status [0, 2]
current angle (45, 180)
[0, 2]
current angle (20, 170)
[1, 2]
good work  status [1, 2]
current angle (10, 170)
[1, 2]
 straighten knees vertically status [1, 2]
current angle (45, 175)
[1, 2]
current angle (90, 170)
[1, 2]
current angle (120, 180)
[1, 2]
current angle (165, 165)
[0, 2]
good work status [0, 2]
current angle (180, 180)
[0, 2]
 bend knees down status [0, 2]
current angle (165, 165)
[0, 2]
current angle (120, 160)
[0, 2]
current angle (90, 170)
[0, 2]
current angle (45, 180)
[0, 2]
current angle (20, 170)
[1, 2]
good work  status [1, 2]
current 

In [90]:
status=[]    
for x in jj:
    status.append(excercise["squat"][x]["status"])


In [91]:
status

[0, 2]

In [92]:
scode

[0, 2]

In [94]:
scode!=status

False

In [96]:
[1,2]==[0,2]

False